# Testing overridable computed fields

### Using base computed fields

Does not allow overriding 

In [1]:
from pydantic import BaseModel, computed_field


class Model(BaseModel):
    foo: str
    bar: str

    @computed_field
    @property
    def foobar(self) -> str:
        return self.foo + self.bar


obj = Model(foo="a", bar="b")
print(obj)  # foo='a' bar='b' foobar='ab'

obj = Model(foo="a", bar="b", foobar="c")
print(obj)

foo='a' bar='b' foobar='ab'
foo='a' bar='b' foobar='ab'


### Field validator 

Couldn't get it to work with no `foobar` input

In [2]:
from typing import Any

from pydantic import BaseModel, ValidationInfo, field_validator


class Model(BaseModel):
    foo: str
    bar: str
    foobar: str = None

    @field_validator("foobar", mode="before")
    @classmethod
    def set_if_empty(cls, value: str, model_dump: ValidationInfo) -> str:
        if value is None:
            return model_dump.data["foo"] + model_dump.data["bar"]
        return value


obj = Model(foo="a", bar="b")
print(obj)

obj = Model(foo="a", bar="b", foobar=None)
print(obj)  # foo='a' bar='b' foobar='ab'

obj = Model(foo="a", bar="b", foobar="c")
print(obj)  # foo='a' bar='b' foobar='ab'

foo='a' bar='b' foobar=None
foo='a' bar='b' foobar='ab'
foo='a' bar='b' foobar='c'


### Model validator: after

Can deal with the model as if its a model. foobar must be allowed to take `None` if want to pass `None`

In [3]:
from pydantic import BaseModel, model_validator


class Model(BaseModel):
    foo: str
    bar: str
    foobar: str | None = None

    @model_validator(mode="after")
    def set_if_empty(self) -> str:
        if self.foobar is None:
            self.foobar = self.foo + self.bar

        return self


obj = Model(foo="a", bar="b")
print(obj)

obj = Model(foo="a", bar="b", foobar=None)
print(obj)  # foo='a' bar='b' foobar='ab'

obj = Model(foo="a", bar="b", foobar="c")
print(obj)  # foo='a' bar='b' foobar='ab'

foo='a' bar='b' foobar='ab'
foo='a' bar='b' foobar='ab'
foo='a' bar='b' foobar='c'


### Model validator: before

Don't need to allow foobar to be None, best case for computed fields IMO.
model needs to be dealt with as a dict not a model.

In [4]:
from pydantic import BaseModel, model_validator


class Model(BaseModel):
    foo: str
    bar: str
    foobar: str = None

    @model_validator(mode="before")
    def set_if_empty(model_dump: dict):
        if model_dump.get("foobar") is None:
            model_dump["foobar"] = model_dump["foo"] + model_dump["bar"]

        return model_dump


obj = Model(foo="a", bar="b")
print(obj)  # foo='a' bar='b' foobar='ab'

obj = Model(foo="a", bar="b", foobar=None)
print(obj)  # foo='a' bar='b' foobar='ab'

obj = Model(foo="a", bar="b", foobar="c")
print(obj)  # foo='a' bar='b' foobar='c'

foo='a' bar='b' foobar='ab'
foo='a' bar='b' foobar='ab'
foo='a' bar='b' foobar='c'


In [5]:
from pydantic import BaseModel, model_validator


class Model(BaseModel):
    id: str
    name: str = None

    @model_validator(mode="before")
    def set_if_empty(model_dump: dict):
        if model_dump.get("name") is None:
            model_dump["name"] = model_dump["id"] + "_name"

        return model_dump


obj = Model(id="SA1_2021")
print(obj)  # id='SA1_2021' name='SA1_2021_name'

obj = Model(id="SA1_2021", name=None)
print(obj)  # id='SA1_2021' name='SA1_2021_name'

obj = Model(id="SA1_2021", name="different_name")
print(obj)  # id='SA1_2021' name='different_name'

id='SA1_2021' name='SA1_2021_name'
id='SA1_2021' name='SA1_2021_name'
id='SA1_2021' name='different_name'
